In [1]:
#| default_exp page_seo_audit

In [2]:
#| export
from sqlmodel import Session, select
from seo_rat.sqlite_db import SQLiteDB
from seo_rat.gsc_client import GSCAuth, get_date_range
from seo_rat.gsc_storage import get_top_queries, get_page_analytics, get_trends
from seo_rat.gsc_insights import (
    detect_query_trends,
    classify_page_intents,
    find_missing_queries,
    find_green_keywords,
    get_date_ranges_for_comparison,
)
from seo_rat.content_parser import (
    get_page_content,
    parse_metadata,
    parse_notebook_metadata,
    check_title_length,
    check_desc_length,
    check_content_length,
    extract_headers,
    extract_images,
    extract_links,
    imgs_missing_alts,
)
from seo_rat.seo_content_analysis import check_h1_count, calculate_keyword_density
from seo_rat.content_mapper import map_all_urls_to_files
from seo_rat.index_tracking import fetch_sitemap_urls
from seo_rat.seo_report import analyze_links



In [3]:

#| export
def full_page_audit(
    session: Session,
    site_url: str,
    page_url: str,
    file_path: str,
    domain: str,
    is_quarto: bool = True,
    days: int = 90,
    query_limit: int = 100,
) -> dict:
    """Generate a comprehensive SEO audit for a single page"""

    # --- Content Analysis ---
    content = get_page_content(file_path, is_quarto=is_quarto)
    with open(file_path, "r") as f:
        raw = f.read()

    if file_path.endswith(".ipynb"):
        metadata = parse_notebook_metadata(raw)
    else:
        metadata = parse_metadata(raw)

    title = metadata.get("title", "") or metadata.get("pagetitle", "")
    description = metadata.get("description", "")

    headers = extract_headers(file_path)
    images = extract_images(content)

    basic_seo = {
        "title": title,
        "title_check": check_title_length(title),
        "description": description,
        "description_check": check_desc_length(description),
        "content_check": check_content_length(content),
        "h1_check": check_h1_count(headers, title=title, is_quarto=is_quarto),
        "missing_alts": imgs_missing_alts(images),
        "link_analysis": analyze_links(content, domain),
    }

    # --- GSC Performance ---
    start, end = get_date_range("last_days", days=days)
    page_path = page_url.split(domain)[-1] if domain in page_url else page_url

    gsc_performance = get_page_analytics(session, site_url, page_path, start, end)

    # --- Query Intent ---
    query_intents = classify_page_intents(
        session, site_url, start, end, page_path=page_path, limit=query_limit,
    )

    # --- Trends ---
    trends = detect_query_trends(
        session, site_url, page_path=page_path, days=days, limit=query_limit,
    )
    rising = [t for t in trends if t["trend"] == "rising"]
    declining = [t for t in trends if t["trend"] == "declining"]

    # --- Missing Queries ---
    queries = get_top_queries(
        session, site_url, start, end, page_path=page_path, limit=query_limit,
    )
    missing = find_missing_queries(queries, content)

    # --- Green Keywords ---
    green = find_green_keywords(
        session, site_url, page_path=page_path, content=content, days=days, limit=query_limit,
    )

    return {
        "page_url": page_url,
        "file_path": file_path,
        "basic_seo": basic_seo,
        "gsc_performance": gsc_performance,
        "query_intents": query_intents,
        "trends": {"rising": rising, "declining": declining},
        "missing_queries": missing,
        "green_keywords": green,
    }


In [9]:

#| export
def print_audit(audit: dict):
    """Pretty print a page audit"""
    print(f"{'='*70}")
    print(f"PAGE AUDIT: {audit['page_url']}")
    print(f"{'='*70}")

    # Basic SEO
    seo = audit["basic_seo"]
    print(f"\n--- Basic SEO ---")
    print(f"Title: {seo['title']} ({seo['title_check']['length']} chars, optimal: {seo['title_check']['optimal_length']})")
    print(f"Description: {seo['description'][:60]}... ({seo['description_check']['length']} chars, optimal: {seo['description_check']['optimal_length']})")
    print(f"Content: {seo['content_check']['word_count']} words (sufficient: {seo['content_check']['is_sufficient']})")
    print(f"H1: {seo['h1_check']}")
    print(f"Missing alts: {seo['missing_alts']}")
    print(f"Links: {seo['link_analysis']['internal_count']} internal, {seo['link_analysis']['external_count']} external")

    # GSC Performance
    perf = audit["gsc_performance"]
    print(f"\n--- GSC Performance ---")
    print(f"Clicks: {perf['total_clicks']} | Impressions: {perf['total_impressions']}")
    print(f"Avg Position: {perf['avg_position']:.1f} | Avg CTR: {perf['avg_ctr']:.2%}")

    # Intent Breakdown
    intents = audit["query_intents"]
    if intents:
        from collections import Counter
        intent_counts = Counter(q["intent"] for q in intents)
        for intent, count in intent_counts.most_common():
            print(f"\n  {intent} ({count}):")
            for q in intents:
                if q["intent"] == intent:
                    print(f"    {q['query'][:45]:45} | imp: {q['total_impressions']} | clicks: {q['total_clicks']} | pos: {q['avg_position']:.1f}")


    # Trends
    rising = audit["trends"]["rising"]
    declining = audit["trends"]["declining"]
    print(f"\n--- Trends ---")
    print(f"Rising: {len(rising)} | Declining: {len(declining)}")
    for t in rising[:5]:
        print(f"  ↑ {t['query'][:45]:45} | imp: {t['impression_change']:+d}")
    for t in declining[:5]:
        print(f"  ↓ {t['query'][:45]:45} | imp: {t['impression_change']:+d}")

    # Missing Queries
    missing = audit["missing_queries"]
    print(f"\n--- Missing Queries ({len(missing)}) ---")
    for m in missing[:10]:
        print(f"  {m['query'][:45]:45} | imp: {m['total_impressions']} | clicks: {m['total_clicks']} | pos: {m['avg_position']:.1f}")


    # Green Keywords
    green = audit["green_keywords"]
    print(f"\n--- Green Keywords ({len(green)}) ---")
    for g in green[:10]:
        print(f"  🟢 {g['query'][:45]:45} | {g['prev_impressions']} → {g['recent_impressions']} | pos: {g['recent_position']:.1f}")




In [10]:

#| hide
db = SQLiteDB()
session = db.get_session()
auth = GSCAuth()

urls = fetch_sitemap_urls("https://kareemai.com/sitemap.xml")
url_mapping = map_all_urls_to_files(
    "/home/kobo/Desktop/obsidian_valuts/logseq/karem-site/",
    "https://kareemai.com/",
    urls,
)



In [23]:

#| hide
from urllib.parse import unquote
from seo_rat.gsc_storage import normalize_url

page_url = normalize_url("https://kareemai.com/blog/posts/life_style/fake_graviety.html")


file_path = url_mapping[page_url]

audit = full_page_audit(
    session,
    site_url="sc-domain:kareemai.com",
    page_url=page_url,
    file_path=file_path,
    domain="kareemai.com",
)
print_audit(audit)


PAGE AUDIT: https://kareemai.com/blog/posts/life style/fake graviety.html

--- Basic SEO ---
Title: جاذبية مزيفة و عالم مخادع: كيف تحافظ على بوصلتك؟ (48 chars, optimal: False)
Description: مخاوف عن الوقوع في جاذبية الآخرين وأهوائهم والابتعاد عن الهد... (129 chars, optimal: False)
Content: 2291 words (sufficient: True)
H1: {'h1_count': 1, 'has_single_h1': True, 'h1_source': 'title'}
Missing alts: []
Links: 4 internal, 1 external

--- GSC Performance ---
Clicks: 0 | Impressions: 0
Avg Position: 0.0 | Avg CTR: 0.00%

--- Trends ---
Rising: 0 | Declining: 0

--- Missing Queries (0) ---

--- Green Keywords (0) ---
